In [ ]:
###########################
#   -*- coding: utf-8 -*- #
# PHM_data_challenge_2019 #
#     Author: Huet Zhu    #
#       Date:2019.5       #
#    All Rights Reserved  #
###########################

# # 基于机器学习的飞控系统故障诊断方案设计
from __future__ import division

import numpy as np
import thundergbm
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection  import train_test_split
import numpy as np  

from tqdm import tqdm

import gc

from scipy import sparse
from scipy.sparse import csr_matrix, hstack, vstack

import warnings
import random
warnings.filterwarnings('ignore')

In [ ]:
age_test = pd.read_csv("../data/age_test.csv", header = None)
age_train = pd.read_csv("../data/age_train.csv", header = None)

In [ ]:
csr_trainData = sparse.load_npz('../trainTestData/trainData13100.npz')
csr_trainData = csr_trainData[:, :2100]
csr_trainData.shape

# 随机森林

In [ ]:
k_fold = [[0, int(csr_trainData.shape[0]*0.5)],
          [int(csr_trainData.shape[0]*0.5),  int(csr_trainData.shape[0])],
          [0, int(csr_trainData.shape[0]*0.5)],
          [int(csr_trainData.shape[0]*0.5),  int(csr_trainData.shape[0])],
          [0, int(csr_trainData.shape[0]*0.5)],
          [int(csr_trainData.shape[0]*0.5),  int(csr_trainData.shape[0])],]
# k_fold = [[0, int(csr_trainData.shape[0]*0.2)],
#           [int(csr_trainData.shape[0]*0.2), int(csr_trainData.shape[0]*0.4)], 
#           [int(csr_trainData.shape[0]*0.4), int(csr_trainData.shape[0]*0.6)],
#           [int(csr_trainData.shape[0]*0.6), int(csr_trainData.shape[0]*0.8)],
#           [int(csr_trainData.shape[0]*0.8), int(csr_trainData.shape[0])]]
# k=0
# trainData, valData, trainLabel, valLabel = train_test_split(csr_trainData[k_fold[k][0]:k_fold[k][1]], 
#                                                             age_train.iloc[k_fold[k][0]:k_fold[k][1], 1].values, 
#                                                             test_size=0.1,
#                                                             random_state=k)
# trainData.shape, valData.shape

In [ ]:
k=5
clf = thundergbm.TGBMClassifier(bagging=1, lambda_tgbm=1, learning_rate=0.07, min_child_weight=1.2, n_gpus=1, verbose=0,
                            n_parallel_trees=40, gamma=0.2, depth=7, n_trees=4000, tree_method='auto', objective='multi:softmax')

In [ ]:
clf.fit(csr_trainData[k_fold[k][0]:k_fold[k][1]],  age_train.iloc[k_fold[k][0]:k_fold[k][1], 1].values)
clf.save_model('../model/rf_part_'+str(k)+'.model')

In [ ]:
k=5-k

In [ ]:
clf.score(csr_trainData[k_fold[k][0]:k_fold[k][1]],  age_train.iloc[k_fold[k][0]:k_fold[k][1], 1].values)

In [ ]:
clf.load_model('../model/rf_part_'+str(k)+'.model')
clf.score(csr_trainData[k_fold[k][0]:k_fold[k][1]],  age_train.iloc[k_fold[k][0]:k_fold[k][1], 1].values)

In [ ]:
predict_sigal = clf.predict(csr_trainData[k_fold[k][0]:k_fold[k][1]])

# stacking 数据准备

In [ ]:
trainData = sparse.load_npz('../trainTestData/csr_trainData13100.npz')
testData = sparse.load_npz('../trainTestData/csr_testData13100.npz')
trainData = trainData[:, :2100]
testData = testData[:, :2100]
trainData.shape, testData.shape

In [ ]:
clf = thundergbm.TGBMClassifier(bagging=1, lambda_tgbm=1, learning_rate=0.07, min_child_weight=1.2, n_gpus=1, verbose=0, 
                            n_parallel_trees=40, gamma=0.2, depth=7, n_trees=4000, tree_method='auto', objective='multi:softmax')

In [ ]:
k_fold = [[0, int(trainData.shape[0]*0.5)],
          [int(trainData.shape[0]*0.5),  int(trainData.shape[0])],
          [0, int(trainData.shape[0]*0.5)],
          [int(trainData.shape[0]*0.5),  int(trainData.shape[0])],
          [0, int(trainData.shape[0]*0.5)],
          [int(trainData.shape[0]*0.5),  int(trainData.shape[0])],]
k_flod_predict = np.zeros((trainData.shape[0], 3))
for k in tqdm(range(6)):
    valData = trainData[k_fold[5-k][0]:k_fold[5-k][1]]
    clf.load_model('../model/rf_part_'+str(k)+'.model')
    predict_sigal = clf.predict(valData)

    k_flod_predict[k_fold[5-k][0]:k_fold[5-k][1], k//2] = predict_sigal

In [ ]:
k_flod_predict = k_flod_predict.astype(int)
k_flod_predict

In [ ]:
predict = np.zeros((k_flod_predict.shape[0]))
for i in range(k_flod_predict.shape[0]):
    predict[i] = np.argmax(np.bincount(k_flod_predict[i]))

In [ ]:
predict = predict.astype(int)
predict

In [ ]:
train_stacking_predict = np.zeros((k_flod_predict.shape[0], 6))
for i in range(train_stacking_predict.shape[0]):
    train_stacking_predict[i, predict[i]-1] = 1

In [ ]:
np.savetxt('../processed/stacking/rf_val.txt', train_stacking_predict, fmt='%s', delimiter=',', newline='\n')

# 预测

In [ ]:
testData = sparse.load_npz('../trainTestData/testData13100.npz')
testData = testData[:, :2100]
testData.shape

In [ ]:
clf = thundergbm.TGBMClassifier(bagging=1, learning_rate=0.1, min_child_weight=1.2, n_gpus=1,
                            n_parallel_trees=40, gamma=0.2, depth=7, n_trees=3000, tree_method='auto', objective='multi:softmax')

In [ ]:
k_flod_predict = np.zeros((testData.shape[0], 6))
for i in tqdm(range(6)):
    clf.load_model('../model/rf_part_'+str(i)+'.model')
    predict_sigal =clf.predict(testData)
    for j in range(len(predict_sigal)):
        k_flod_predict[j, i] = predict_sigal[j]

In [ ]:
k_flod_predict = k_flod_predict.astype(int)
k_flod_predict

In [ ]:
predict = np.zeros((k_flod_predict.shape[0]))
for i in range(k_flod_predict.shape[0]):
    predict[i] = np.argmax(np.bincount(k_flod_predict[i]))
predict = predict.astype(int)

In [ ]:
np.savetxt('../processed/stacking/rf_test.txt', test_stacking_predict, fmt='%s', delimiter=',', newline='\n')